# X-Plane 11 Aircraft Taxi Controller

In this notebook, you will implement a controller for the aircraft taxi problem and observe it in action!

First, let's import some of the python packages we will need:

In [ ]:
import time
import numpy as np

from xpc3 import *
from xpc3_helper import *
from plotting_helper import *

Next, we need to get X-Plane 11 ready to go. We will follow the same steps as we did in the X-Plane 11 tutorial notebook:

1. **Open X-Plane 11.** Look for the X-Plane 11 icon in the dock at the bottom of your screen (it should be towards the right). Click this icon and X-Plane 11 will open. A window may pop up saying that there is an update available. If this happens, you can just click "ignore".
2. From the main menu of X-Plane 11, click "New Flight". This should bring up a window with some flight configuration options.
3. In the AIRCRAFT section, select the Cessna Skyhawk.
4. In the location section, select Grant Co Intl (ID is KMWH).
5. You can leave the weather as clear.
6. In the TIME OF DAY section, select a time between 8AM and 10AM local.
7. Click the "Start Flight" button at the bottom right of the screen. It make take a minute or so to load the flight. Once it loads, you should see the front of the aircraft pointing down a runway. We are now ready to start controlling it!

Connect to X-Plane 11 by creating a client that we can use to interface with the simulator:

In [ ]:
client = XPlaneConnect()
reset(client)

## Simulation Functions
To simulate our taxi controller, we need to implement a *closed-loop system*. This can be a big task, so let's break it down into smaller ones! At each time step, we want to:

1. Get the current state of the system
2. Determine our control input using the state (steering angle)
3. Feed the current state and control input into the dynamics function to determine the next state
4. Go back to step 1!

Let's make a separate function for steps 1-3. The function for step 1 returns the current state and has been done for you:

### 1. Get the current state
This one has been done for you and just uses the data from X-Plane 11 to return the current crosstrack and heading error.

In [ ]:
def getState(client):
    """ Returns the true crosstrack error (meters) and
        heading error, (degrees) to simulate fully 
        oberservable control
        Args:
            client: XPlane Client
    """
    cte, _, he = getHomeState(client)
    return cte, he

### 2. Determine the control input

Remember all of that tuning you did on you controller? Let's test out those values here! Complete the function below with the best gains you found for your controller. Remember, the control law is:

    phi = cte_gain * cte + he_gain * he

In [ ]:
def getControl(client, cte, he):
    """ Returns steering angle command using proportional control
        Args:
            client: XPlane Client
            cte: current estimate of the crosstrack error, x (meters)
            he: current estimate of the heading error, theta (degrees)
    """
    # STUDENT CODE START
    
    # STUDENT CODE END

### 3. Get the next state
Fill in the dynamics function to return the next state. Remember the dynamics equations we found:
- he_next = he + phi * dt
- cte_next = cte + v * sin(theta) * dt
- dtp_next = dtp + v * cos(theta) * dt

For sin and cos, you can use the `np.sin` and `np.cos` functions. NOTE: these functions expect the input angle to be in RADIANS. Just like fahrenheit and celsius are different ways of measuring temperature, degrees and radians are different ways to measure angles! And just like we can convert a temperature from fahrenheit to celsius and vice versa, we can convert from degrees to radians and vice versa. Numpy has a nice function to do this for us called `np.deg2rad`. We have created variables for the radians values of the heading error for you (`he_rad`). **TLDR:** use the radians value (`he_rad`) when calling the sin and cos functions :)

In [ ]:
def dynamics(cte, dtp, he, phi_deg, dt=0.05, v=5):
    """ dynamics model (returns next state)
        Args:
            cte: current crosstrack error (meters)
            dtp: current downtrack position (meters)
            he: current heading error (degrees)
            phi_deg: steering angle input (degrees)
            -------------------------------
            dt: time step (seconds)
            v: speed (m/s)
    """

    he_rad = np.deg2rad(he)

    # STUDENT CODE START
    
    # STUDENT CODE END

    return cte_next, he_next, dtp_next

### Put it all together!
We can now put everything together into a single simulation function. We have started by intializing some variables for you. Your job is to fill in the for loop that will simulate the closed-loop system. Here is a rough outline of the steps your code should follow:

1. **Store the current crosstrack error, heading error, and downtrack position**. We want to keep track of the trajectory of the aircraft during our simulation so that we can plot it and analyze it after. To do this, we have defined three arrays for you: `cte_history`, `he_history`, and `dtp_history`. To start each loop iteration, you should store the values of these variables at the next slot in the array.
2. **Decide if we should update our control**. Most real-world systems operate at a specific frequency, meaning that they only update their control every so often. For our simulation, we will only update our control (steering angle) on certain iterations of the loop controlled by the `ctrl_every` input. The default for this input is 20, so we only want to update our control at iteration 0, 20, 40, 60, 80, etc. What condition can we use to determine this? HINT: check out (python's modulo operator)[https://www.freecodecamp.org/news/the-python-modulo-operator-what-does-the-symbol-mean-in-python-solved/]. Can you come up with a condition using this operator that would only be true when i is a multiple of `ctrl_every`?
3. **If we should update our control, update it**. To do this, we need to get the cte and he using the `getState` function and then input those into the `getControl` function to get a new value for `phiDeg`.
4. **Get the next state based on the dynamics**. Call your dynamics function with the appropriate inputs.
5. **Tell the simulator to move to that state**. You can use the function `setHomeState(client, cte, dtp, he)`

In [ ]:
def simulate_controller(client, startCTE, startHE, startDTP, getState, getControl,
                               dt=0.05, ctrlEvery=20, nsteps = 400, simSpeed=1.0):
    """ Simulates a controller, overriding the built-in XPlane-11 dynamics to model the aircraft
        as a Dubin's car
        Args:
            client: XPlane Client
            startCTE: Starting crosstrack error (meters)
            startHE: Starting heading error (degrees)
            startDTP: Starting downtrack position (meters)
            getState: Function to estimate the current crosstrack and heading errors.
                      Takes in an XPlane client and returns the crosstrack and
                      heading error estimates
            getControl: Function to perform control based on the state
                        Takes in an XPlane client, the current crosstrack error estimate,
                        and the current heading error estimate and returns a control effort
            -------------------
            dt: time step (seconds)
            crtlEvery: Frequency to get new control input 
                       (e.g. if dt=0.5, a value of 20 for ctrlEvery will perform control 
                       at a 1 Hz rate)
            nsteps: Number of time steps to run the simulation for
            simSpeed: increase beyond 1 to speed up simulation
    """
    # Reset to the desired starting position
    client.sendDREF("sim/time/sim_speed", simSpeed)
    reset(client, cteInit=startCTE,
                      heInit=startHE, dtpInit=startDTP)
    sendBrake(client, 0)

    time.sleep(5)  # 5 seconds to get terminal window out of the way

    # Initialize the crosstrack error, heading error, and downtrack position
    cte = startCTE
    he = startHE
    dtp = startDTP

    # Initialize the steering angle to 0 degrees
    phiDeg = 0.0 # degrees

    # Initialize arrays of zeros to store the results in
    cte_history = np.zeros(nsteps)
    he_history = np.zeros(nsteps)
    dtp_history = np.zeros(nsteps)

    # STUDENT CODE START
    for i in range(nsteps): # Repeat for nsteps
        # Record the current cte, he, and dtp into their corresponding arrays

        # If we should change our control

        # Update the cte, he, and dtp using the dynamics function

        # Set the simulator to move to the next state

        # Sleep for 0.03 seconds to give the simulator time to update everything
        time.sleep(0.03)
    # STUDENT CODE END

    return cte_history, he_history, dtp_history

## Test your controller!

### Specify the settings

The cell below defines some settings for the simulator such as the time of day and weather conditions. You can just leave these as is for now.

In [ ]:
# Time of day in local time, e.g. 8.0 = 8AM, 17.0 = 5PM
TIME_OF_DAY = 8.0

# Cloud cover (higher numbers are cloudier/darker)
# 0 = Clear, 1 = Cirrus, 2 = Scattered, 3 = Broken, 4 = Overcast
CLOUD_COVER = 0

# Start downtrack position (322 m is a good starting place)
START_DTP = 322.0

# Set weather and time of day
client.sendDREF("sim/time/zulu_time_sec", TIME_OF_DAY * 3600 + 8 * 3600)
client.sendDREF("sim/weather/cloud_type[0]", CLOUD_COVER)

### Run a single simulation
The line below will run a single simulation and store the results in arrays called `cte_history`, `he_history`, and `dtp_history`. Run this cell now to watch your controller in action! It will pause for 5 seconds before starting and then run for 20 seconds.

In [ ]:
cte_history, he_history, dtp_history = simulate_controller(client, 6.0, 0.0, START_DTP, getState, getControl)

### Plot the results

Sometime, it is helpful to visualize the trajectory that the aircraft followed. We can use this to see if the aircraft is behaving how we expect it to. Also, we can use it to show others how our system performs (like in your final presentation!). The following two cells contain code for you to do this. The first cell imports some python plotting packages that you will use. The second cell plots the trajectory on the road. We have implemented a function for you to get a plot of the road in the `plotting_helper.py` file.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

In [ ]:
# Get the plot of the road
fig, ax = plot_road()
# Plot the trajectory on the road
ax.plot(dtp_history - START_DTP, cte_history, 'b', linewidth=1)
# Add axis labels
ax.set_xlabel('Downtrack Position (m)')
ax.set_ylabel('Crosstrack Error (m)')
# Display the plot
plt.show()

### Run many simulations

Now that we have confirmed that our controller is working as intended for one scenario, let's test out some other scenarios. It is important that we make sure our controller will work in many different scenarios. To test this, your next task is to write some code to find the trajectory of the aircraft from a number of starting crosstrack errors by filling in the code cells below.

First, let's initialize some lists to store our outputs:

In [ ]:
cte_histories = []
he_histories = []
dtp_histories = []

Next, let's call our simulation function from a number of ctes and append the resuts to our lists:

In [ ]:
start_ctes = [-8.0, -6.0, -4.0, -2.0, 0.0, 2.0, 4.0, 6.0, 8.0]

for start_cte in start_ctes:
    # STUDENT CODE START
    # Simulate the trajectory and get the results
    
    # Append to the lists

    # STUDENT CODE END

Finally, let's plot the results! Our goal is to generate a plot similar to the one above for the single trajectory but with all of the trajectories we tested on it. You can use similar code to the plotting code used for the single trajectory. For this case, instead of adding a single trajectory, you will want to loop through the histories and add each trajectory to the plot.

In [ ]:
# STUDENT CODE START
# Get the plot of the road

# Plot the trajectory on the road

# Add axis labels

# STUDENT CODE END

# Display the plot
plt.show()